In [1]:
import numpy as np

#Store Data Variables
import json
with open('feature_data.json', 'r') as f:
 features = json.load(f)

from scipy.io import loadmat
train_idxs = loadmat('cuhk03_new_protocol_config_labeled.mat')['train_idx'].flatten()
query_idxs = loadmat('cuhk03_new_protocol_config_labeled.mat')['query_idx'].flatten()
labels = loadmat('cuhk03_new_protocol_config_labeled.mat')['labels'].flatten()
gallery_idxs = loadmat('cuhk03_new_protocol_config_labeled.mat')['gallery_idx'].flatten()
filelist = loadmat('cuhk03_new_protocol_config_labeled.mat')['filelist'].flatten()
camId = loadmat('cuhk03_new_protocol_config_labeled.mat')['camId'].flatten()

In [2]:
train_idxs

array([    1,     2,     3, ..., 14094, 14095, 14096], dtype=uint16)

In [3]:
query_idxs

array([   22,    27,    49, ..., 14043, 14059, 14063], dtype=uint16)

In [4]:
labels

array([   1,    1,    1, ..., 1467, 1467, 1467], dtype=uint16)

In [5]:
gallery_idxs

array([   21,    23,    24, ..., 14062, 14064, 14065], dtype=uint16)

In [6]:
filelist

array([array(['1_001_1_01.png'], dtype='<U14'),
       array(['1_001_1_02.png'], dtype='<U14'),
       array(['1_001_1_03.png'], dtype='<U14'), ...,
       array(['5_049_2_08.png'], dtype='<U14'),
       array(['5_049_2_09.png'], dtype='<U14'),
       array(['5_049_2_10.png'], dtype='<U14')], dtype=object)

In [7]:
camId

array([1, 1, 1, ..., 2, 2, 2], dtype=uint8)

In [8]:
#grid searc cv against k, distance metric and transformation/normalization - use only query and gallery for this part 

In [9]:

X = np.array(features)
y = np.array(labels)
filelist = np.array(filelist)
camId = np.array(camId)



In [10]:
mask_train = np.array(train_idxs).ravel()
mask_query = np.array(query_idxs).ravel()
mask_gallery = np.array(gallery_idxs).ravel()

mask_train = np.subtract(mask_train, 1)
mask_query = np.subtract(mask_query, 1)
mask_gallery = np.subtract(mask_gallery, 1)


X_train, X_query, X_gallery = X[mask_train, :], X[mask_query, :], X[mask_gallery, :]
y_train, y_query, y_gallery = y[mask_train], y[mask_query], y[mask_gallery]
filelist_train, filelist_query, filelist_gallery = filelist[mask_train], filelist[mask_query], filelist[mask_gallery]
camId_train, camId_query, camId_gallery = camId[mask_train], camId[mask_query], camId[mask_gallery]


In [43]:
from scipy.spatial import distance

rank_accuracies = []

#q = 0
#print (X_query.shape)

for query, camId_q, y_q in zip(X_query, camId_query, y_query):
    q_g_dists = []
    y_valid = []
    for gallery, camId_g, y_g  in zip(X_gallery, camId_gallery, y_gallery):
        if ((camId_q == camId_g) and (y_q == y_g)):
            continue
        else:
            dist = distance.euclidean(query, gallery)
            q_g_dists.append(dist)
            y_valid.append(y_g)
    
    q_g_dists = np.array(q_g_dists)
    y_valid = np.array(y_valid)
    
    _indexes = np.argsort(q_g_dists)
    
    # Sorted distances and labels
    q_g_dists, y_valid = q_g_dists[_indexes], y_valid[_indexes]
    
    
    rank_1  = 0
    rank_5  = 0
    rank_10 = 0
    rank_20 = 0  
    
    for k in range (20):
        
        if (y_valid[k] == y_q):
            if (k < 1):
                rank_1  = 1
            if (k < 5):
                rank_5 = 1
            if (k < 10):
                rank_10 = 1
            if (k < 20):
                rank_20 = 1
    
    rank_accuracies.append((rank_1, rank_5, rank_10, rank_20))
    
#    if q  > 20:
#        break
#    q = q+1
    

    
rank_accuracies = np.array(rank_accuracies)

total = rank_accuracies.shape[0]
rank_accuracies = rank_accuracies.sum(axis = 0)
rank_accuracies = np.divide(rank_accuracies, total)

print (rank_accuracies)

#rank_accuracies = np.array([[1, 0, 0, 0],
# [1, 0, 0, 0],
# [0, 1, 0, 0],
# [0, 0, 1, 1],
# [0, 0, 0, 0]])
#print(rank_accuracies.shape)
#print(rank_accuracies)
#total = rank_accuracies.shape[0]
#rank_accuracies = rank_accuracies.sum(axis = 0)
#rank_accuracies = np.divide(rank_accuracies, total)
#print(rank_accuracies.shape)
#print(rank_accuracies)
    
        
    

[0.47       0.66857143 0.74928571 0.83214286]
